In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
from matplotlib import pyplot as plt
import pandas as pd

import sys
sys.path.append("../")
from bargains_linear import * 

In [2]:
def gen_data(params,nobs=100,var=5):
    wtp,cost,beta1,beta2,emc1,emc2=params
    emc = emc1,emc2
    passive_result = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2']) 
    active_result = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2'])
    seq_result = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2'])

    for i in range(0,nobs):
        #sequential
        mc1 = np.random.normal(loc=0, scale=var, size=(2,)) + np.array([emc1,emc2])
        mc2 = np.random.normal(loc=0, scale=1, size=(2,))
        mc = mc1+mc2

        #seq results
        phi1,phi2 = seq_bargain(25,cost,wtp,mc)
        phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp,mc)
        row = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2'],data=[[phi1,phi2,p1,p2,s1,s2,mc1[0],mc1[1]]])
        seq_result = pd.concat( [seq_result, row] )


        #active results
        phi1,phi2 =simult_bargain(25,25, cost,wtp,mc, active=True)
        phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp,mc)
        row = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2'],data=[[phi1,phi2,p1,p2,s1,s2,mc1[0],mc1[1]]])
        active_result = pd.concat( [active_result, row] )


        #passive results
        phi1,phi2 =simult_bargain(25,25, cost,wtp,mc, active=False)
        phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp,mc)
        row = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2'],data=[[phi1,phi2,p1,p2,s1,s2,mc1[0],mc1[1]]])
        passive_result = pd.concat( [passive_result, row] )
    return seq_result, active_result,passive_result


seq_result,active_result,passive_result = gen_data([25,5,.5,.5,0,0],nobs=1000,var=5)
print(seq_result)
seq_result.to_csv('fake_data/seq_data.csv')
active_result.to_csv('fake_data/active_data.csv')
passive_result.to_csv('fake_data/passive_data.csv')

         phi1       phi2         p1         p2        s1        s2         c1  \
0   15.011153  19.007949  25.929300  26.933144  0.600384  0.399616   5.217947   
0   13.622042  18.173819  22.652371  23.100953  0.544858  0.455142   3.399318   
0   31.200380  32.275283  28.263831  32.188916  0.892509  0.107491 -12.371471   
0   16.248802  19.751097  21.280359  22.778498  0.649814  0.350186  -1.721989   
0   32.302309  33.101735  32.314188  36.514773  0.920059  0.079941  -7.988026   
..        ...        ...        ...        ...       ...       ...        ...   
0   28.214511  30.114656  28.589994  31.689846  0.809985  0.190015  -7.303571   
0   29.884680  31.288492  35.527043  39.123196  0.859615  0.140385  -2.132428   
0   12.719432  17.632032  23.928301  24.016006  0.508770  0.491230   5.368191   
0   11.194562  16.716872  16.208169  15.686016  0.447785  0.552215  -0.593458   
0   14.763756  18.859357  13.984382  14.889347  0.590496  0.409504  -8.046165   

          c2  
0   2.848431

In [3]:
#calculate moment conditions for the linear model...
def solve_parameters_obj(data,params, active=False,seq=False):
    wtp,cost,beta1,beta2,mc1,mc2 = params
    
    data_moments = list(data.mean())
    phi1,phi2 = None,None
    if seq:
        phi1,phi2 = seq_bargain(25, cost, wtp, [mc1,mc2],betas=[beta1,beta2])
    elif active:
        phi1,phi2 = simult_bargain(25,25, cost, wtp, [mc1,mc2],active=True,betas=[beta1,beta2])
    else:
        phi1,phi2 = simult_bargain(25,25, cost, wtp, [mc1,mc2],active=False,betas=[beta1,beta2])
    
    phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp,[mc1,mc2])

    computed_moments = [phi1,phi2,p1,p2,s1,s2]
    
    obj = 0
    for i in range(6):
        obj = obj + (data_moments[i] - computed_moments[i])**2
    
    return obj

print(seq_result)

print( solve_parameters_obj(seq_result,[25,5,.5,.5,0,0], seq=True) )
print( solve_parameters_obj(seq_result,[25,5,.5,.5,0,0], active=True) )

         phi1       phi2         p1         p2        s1        s2         c1  \
0   15.011153  19.007949  25.929300  26.933144  0.600384  0.399616   5.217947   
0   13.622042  18.173819  22.652371  23.100953  0.544858  0.455142   3.399318   
0   31.200380  32.275283  28.263831  32.188916  0.892509  0.107491 -12.371471   
0   16.248802  19.751097  21.280359  22.778498  0.649814  0.350186  -1.721989   
0   32.302309  33.101735  32.314188  36.514773  0.920059  0.079941  -7.988026   
..        ...        ...        ...        ...       ...       ...        ...   
0   28.214511  30.114656  28.589994  31.689846  0.809985  0.190015  -7.303571   
0   29.884680  31.288492  35.527043  39.123196  0.859615  0.140385  -2.132428   
0   12.719432  17.632032  23.928301  24.016006  0.508770  0.491230   5.368191   
0   11.194562  16.716872  16.208169  15.686016  0.447785  0.552215  -0.593458   
0   14.763756  18.859357  13.984382  14.889347  0.590496  0.409504  -8.046165   

          c2  
0   2.848431

In [4]:
def solve_parameters(data,active=False,seq=False): 
    obj = lambda params: solve_parameters_obj(data,params,active=active,seq=seq)
    params = minimize(obj, [25,5,.5,.5,0,0], options={'fatol':1e-2,'xatol':1e-2,'maxiter':20},method='Nelder-Mead').x # 
    return params

print(solve_parameters(seq_result,active=False,seq=True))
print(solve_parameters(active_result,active=True,seq=False))
print(solve_parameters(passive_result,active=False,seq=False))

[2.60916064e+01 4.93665620e+00 4.55787041e-01 5.37750773e-01
 1.50286136e-04 8.26004416e-05]
[ 2.43697653e+01  5.08192457e+00  5.12346410e-01  5.16633090e-01
 -1.07189374e-04  3.55385465e-05]
[2.44701369e+01 5.05316308e+00 4.96905201e-01 5.04297775e-01
 5.52770827e-05 2.01104037e-04]
